If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [ ]:
!zip -r /content/bart-base-two-sentence-horror/checkpoint-2000 /content/bart-base-two-sentence-horror

  adding: content/bart-base-two-sentence-horror/ (stored 0%)
  adding: content/bart-base-two-sentence-horror/runs/ (stored 0%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-07-11_5b6d85d648ff/ (stored 0%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-07-11_5b6d85d648ff/1681070837.322833/ (stored 0%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-07-11_5b6d85d648ff/1681070837.322833/events.out.tfevents.1681070837.5b6d85d648ff.608.3 (deflated 62%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-07-11_5b6d85d648ff/events.out.tfevents.1681070837.5b6d85d648ff.608.2 (deflated 63%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-02-51_5b6d85d648ff/ (stored 0%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-02-51_5b6d85d648ff/events.out.tfevents.1681070775.5b6d85d648ff.608.0 (deflated 63%)
  adding: content/bart-base-two-sentence-horror/runs/Apr09_20-02-51_5b6d85d648ff/1681070775.148916/ (stored 0%)
  add

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numpy requests nlpaug

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split 
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import transformers

print(transformers.__version__)

4.27.4


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [XSum dataset](https://arxiv.org/pdf/1808.08745.pdf) (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

![Widget inference on a summarization task](https://github.com/huggingface/notebooks/blob/main/examples/images/summarization.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [ ]:
model_checkpoint = "facebook/bart-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric, Dataset
metric = load_metric("rouge")

<ipython-input-9-16639867fafc>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
data = pd.read_csv("nov_thru_present.csv")
data.drop('Unnamed: 0', axis=1, inplace = True)
data.rename({'selftext': 'body'}, axis = 1, inplace = True)

titleLen = data['title'].str.split(' ').apply(len)
bodyLen = data['body'].dropna().str.split(' ').apply(len)
over100 = list(bodyLen[bodyLen > 100].index)

data = data.drop(over100, axis=0)

emptyBodies = data[data['body'].isna()]['title']
titleSentLens = emptyBodies.str.split('.').apply(len)
data2 = data.drop(list(data[data['body'].isna()].index), axis=0) 

emptyBodiesSplit = emptyBodies.str.split('.')
# find the indices of posts with exactly two sentences
sentences = emptyBodiesSplit[emptyBodiesSplit.apply(len) == 2]
# separate the sentences
sent1 = sentences.apply(lambda x: x[0])
sent2 = sentences.apply(lambda x: x[1])
# check for empty sent2
# this occurs when there is only one sentence with a period at the end so splitting on a 
# period will return a list of length 2 but the second element is the empty string
# we also check for greater than length 1 in case the period is inside of a quote
print('There are', sum(sent2.apply(len) > 1), 'non-empty second sentences')

sent1 = sent1[sent2.apply(len) > 1]
sent2 = sent2[sent2.apply(len) > 1]

fixed_two_sent = pd.DataFrame({'title': sent1, 'body': sent2})
data = pd.concat([data2, fixed_two_sent], axis=0)
data = data.reset_index(drop = True)

data

There are 150 non-empty second sentences


,title,body
0,"You, a tornado chaser, are looking at a tornad...","Then suddenly, it reverses direction."
1,"My daddy always told me ""If you got nothing go...","And as my abusive spouse screamed, bargained, ..."
2,They say that it takes a while for the pain of...,I’ve just been hit with a full spread of bucks...
3,My co-worker pranked me in the elevator by pre...,I watched in horror as the elevator screen cou...
4,"""I have finally become a beautiful bride of a ...","Shockingly, Mira's husband died the next day a..."
...,...,...
20595,"That unmistakable sound, like a watermelon bei...",How could they do this to my boy?
20596,I was casually taking a sip of soda,It was too late when I realised it was scary ...
20597,Fifty-seven humans were captured for an alien ...,"""I want to connect them all together by the h..."
20598,"I looked back at her, trying to mimic her move...",If only she knew what was on the other side o...


In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path=model_dir+'GoogleNews-vectors-negative300.bin',
    action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path=model_dir+'GoogleNews-vectors-negative300.bin',
    action="insert")


data['title_aug'] = []

In [ ]:
first_split, test = train_test_split(data, test_size=.2)
train, validation = train_test_split(first_split, test_size=.1)

In [ ]:
train_ds = Dataset.from_pandas(train, split='train')
test_ds = Dataset.from_pandas(test, split='test')
validation_ds = Dataset.from_pandas(validation, split='validation')

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
# # https://huggingface.co/docs/datasets/v2.11.0/en/tabular_load 
# hf_data = Dataset.from_pandas(data)

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['title']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["body"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
# preprocess_function(hf_data[:2])

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
train_tokenized_datasets = train_ds.map(preprocess_function, batched=True)
test_tokenized_datasets = test_ds.map(preprocess_function, batched=True)
validation_tokenized_datasets = validation_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/14832 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-two-sentence-horror",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=validation_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.299546,14.857400,1.955700,12.516800,12.580300,17.862900
2,3.652500,3.257668,14.971900,1.996600,12.614600,12.678400,18.057600
3,3.409900,3.247810,15.263700,2.072400,12.869400,12.911900,18.325800
4,3.303200,3.240164,15.191600,2.084800,12.778000,12.849400,18.135300
5,3.233700,3.240542,15.095900,2.040500,12.712800,12.764200,18.129200


TrainOutput(global_step=2320, training_loss=3.373358259529903, metrics={'train_runtime': 752.3675, 'train_samples_per_second': 98.569, 'train_steps_per_second': 3.084, 'total_flos': 1892498113658880.0, 'train_loss': 3.373358259529903, 'epoch': 5.0})

In [ ]:
latest_checkpoint = "bart-base-two-sentence-horror/checkpoint-2000"

trained_model = AutoModelForSeq2SeqLM.from_pretrained(latest_checkpoint).to('cpu')
trained_tok = AutoTokenizer.from_pretrained(latest_checkpoint)

In [ ]:
my_pipe = transformers.Text2TextGenerationPipeline(trained_model, tokenizer=tokenizer)

In [ ]:
my_pipe.model.config.max_length = 1024

In [ ]:
my_pipe.model.config.repetition_penalty = 4

In [ ]:
my_pipe("I went skydiving.")

[{'generated_text': "Then I realized I wasn't skydiving."}]

In [ ]:
my_pipe("I checked myself in the mirror.")

[{'generated_text': 'I checked myself in the mirror.'}]

In [ ]:
my_pipe("The woman pushed a stroller with a cute little chihuahua in it, and I couldn’t help but stop to pet it.")

[{'generated_text': 'It’s been a week now, and I can’t wait to see how it’ll react when I get home.'}]

In [ ]:
my_pipe('My Grand Mother always said that good people die old.')

[{'generated_text': 'I don’t know what she meant, but I’m starting to think she’s right.'}]

In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path=model_dir+'GoogleNews-vectors-negative300.bin',
    action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```